<a href="https://colab.research.google.com/github/spaziochirale/TinyML/blob/master/SC_arduino_tinyml_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.arduino.cc/"><img src="https://raw.githubusercontent.com/sandeepmistry/aimldevfest-workshop-2019/master/images/Arduino_logo_R_highquality.png" width=200/></a>
# Tiny ML su Arduino
## Riconoscimento delle Gesture
Versione tradotta dal lavoro originale di:

 * Sandeep Mistry - Arduino
 * Don Coleman - Chariot Solutions


https://github.com/arduino/ArduinoTensorFlowLiteTutorials/

## Predisposizione dell'ambiente di esecuzione

Google Colaboratory ha già tutte le librerie Python che ci servono, installate, per cui non è necessario scaricare alcun package.
L'istruzione che segue, si limita ad installare sul sistema operativo l'utility Unix "**xdd**" che potrebbe non essere presente.

In [ ]:
# Setup environment
!apt-get -qq install xxd

# Carichiamo i dati campionati tramite Arduino

1. Sulla sinistra di questa pagina web è presente un menù a colonna con diverse icone
1. Selezionare l'icona a forma di "cartella" per aprire l'area che rappresenta la cartella '/content' del server che ospita questo notebook.
1. Trascinare i file  `punch.csv` e `flex.csv` dal computer su quest'area per caricarli all'interno dell'ambiente colab.

I file rimarranno registrati sul server finché la sessione corrente è attiva. Se viene persa la connessione con il server colab oppure viene chiusa la pagina del browser, lo spazio virtuale allocato sarà recuperato dal sistema, i file trasferiti non ci saranno più, e occorrerà ricaricarli dopo aver avviato la nuova sessione di lavoro.

# Visualizziamo graficamente i dati (opzionale)

Costruiamo due grafici separati per i campioni ricavati dall'accelerometro e dal giroscopio, poiché i due insiemi di dati hanno unità di misura e scala di valori differenti.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# lista delle gesture che abbiamo campionato
GESTURES = [
    "punch",
    "flex",
]
NUM_GESTURES = len(GESTURES)
for gesture_index in range(NUM_GESTURES):
  print("Gesture:",GESTURES[gesture_index])
  filename = "/content/" + gesture + ".csv"

  df = pd.read_csv(filename)

  index = range(1, len(df['aX']) + 1)

  plt.rcParams["figure.figsize"] = (20,10)

  plt.plot(index, df['aX'], 'g', label='x', linestyle='solid')
  plt.plot(index, df['aY'], 'b', label='y', linestyle='solid')
  plt.plot(index, df['aZ'], 'r', label='z', linestyle='solid')
  plt.title("Acceleration")
  plt.xlabel("Sample #")
  plt.ylabel("Acceleration (G)")
  plt.legend()
  plt.show()

  plt.plot(index, df['gX'], 'g', label='x', linestyle='solid')
  plt.plot(index, df['gY'], 'b', label='y', linestyle='solid')
  plt.plot(index, df['gZ'], 'r', label='z', linestyle='solid')
  plt.title("Gyroscope")
  plt.xlabel("Sample #")
  plt.ylabel("Gyroscope (deg/sec)")
  plt.legend()
  plt.show()


# Addestriamo la Rete Neurale





## Parsing e preparazione dei dati

Le istruzioni che seguono effettuano il parsing dei dati csv e li trasformano in un formato che sarà utilizzato per addestrare la rete neurale di tipo "denso" o "fully connected".

La lista `GESTURES` viene aggiornata con i dati raccolti nel formato `.csv`.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print(f"Versione della libreria TensorFlow = {tf.__version__}\n")

# Impostiamo il valore del seed della funzione random ad un valore fisso
# in questo modo ogni volta che eseguiamo questo notebook genereremo gli stessi numeri
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# lista delle gesture che abbiamo campionato
GESTURES = [
    "punch",
    "flex",
]

SAMPLES_PER_GESTURE = 119

NUM_GESTURES = len(GESTURES)

# generiamo una codifica one-hot per rappresentare l'output della rete
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# leggiamo ciascun file csv file e inseriamo i valori corrispondenti negli array inputs e outputs
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Elaboro l'indice {gesture_index} per la gesture '{gesture}'.")

  output = ONE_HOT_ENCODED_GESTURES[gesture_index]

  df = pd.read_csv("/content/" + gesture + ".csv")

  # calcoliamo il numero di esempi per le gesture presenti nei file
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)

  print(f"\tCi sono {num_recordings} registrazioni della gesture {gesture}.")

  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      # normalizziamo i dati di input tra 0 e 1:
      # - i valori di accelerazione variano tra: -4 e +4
      # - i valori del giroscopio variano tra: -2000 e +2000
      tensor += [
          (df['aX'][index] + 4) / 8,
          (df['aY'][index] + 4) / 8,
          (df['aZ'][index] + 4) / 8,
          (df['gX'][index] + 2000) / 4000,
          (df['gY'][index] + 2000) / 4000,
          (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)

# convertiamo le liste nel tipo dato numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print("Preparazione dei dati completata.")

## Prepariamo le coppie input-output per il training mescolando i dati in modo casuale

Suddividiamo in modo casuale le coppie input-output separandole in tre insiemi: 60% dei campioni per il training, 20% per la validazione, e 20% per il testing.

  - l'insieme di training viene usato per addestrare la rete
  - l'insieme di validazione viene usato per misurare le prestazioni del processo di addestramento durante la fase di training
  - l'insieme di test viene usato per valutare la rete dopo che il processo è terminato

In [ ]:
# Mescolo in modo casuale i dati di input in modo da preparare in modo
# statisticamente più significativo gli insiemi usati per training, validazione e test
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Scambio gli indici consecutivi (0, 1, 2, etc) con i valori di indice randomizzati
inputs = inputs[randomize]
outputs = outputs[randomize]

# Suddivido le registrazioni (gruppi di dati campionati) in tre diversi insiuemi: training, testing e validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Mescolamento e separazione dei campioni completata.")
print(f"Ci sono {len(inputs_train)} registrazioni nel set di training, {len(inputs_validate)} registrazioni nel set di validation e {len(inputs_test)} in quello di test")

In [ ]:
inputs_train.shape

## Costruiamo la rete e procediamo con l'addestramento

Costruiamo e addestriamo un modello (o rete) [TensorFlow](https://www.tensorflow.org) utilizzando le API di alto livello [Keras](https://www.tensorflow.org/guide/keras).

La nostra rete sarà di tipo DNN, con un primo strato di input costituito da 50 neuroni, uno strato intermedio con 15 neuroni e uno strato di output di due neuroni, uno per ciascuna gesture, i cui valori in uscita rappresenteranno la classificazione effettuata dalla rete in termini di valore di probabilità.

In [ ]:
# costruzione del modello (rete neurale) e training
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=1, validation_data=(inputs_validate, outputs_validate))



## Verifica

Visualizziamo un grafico delle prestazioni della rete durante la fase di validazione

### Visualizziamo il grafico della funzione loss

Visualizziamo il grafico del  "*loss*" per vedere dove il modello smette di migliorare.

In [ ]:
# definiamo le dimensioni del grafico in modo che sia più grande del valore di default che altrimenti sarebbe (6,4).
plt.rcParams["figure.figsize"] = (20,10)

# disegnamo i valori della funzione loss, il modello è condfigurato per usare "mean squared error" come loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training e validation loss')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(plt.rcParams["figure.figsize"])

### Visualizziamo nuovamente il loss spostandoci un poco in avanti rispetto al punto iniziale

Visualizziamo lo stesso grafico della cella di codice precedente ma partendo dall'indice 100 in modo da effettuare una sorta di zoom laddove il modello inizia a convergere.

In [ ]:

SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Visualizziamo l'errore medio assoluto

Il [Mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) è un'ulteriore metrica per valutare le prestazioni del modello.



In [ ]:
# disegno i valori mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b', label='Validation MAE')
plt.title('Training e validation mean absolute error')
plt.xlabel('Epoche')
plt.ylabel('MAE')
plt.legend()
plt.show()


### Facciamo girare la rete con i dati di test
Inseriamo nella rete i dati di test e visualizziamo le previsioni


In [ ]:
# usiamo la rete per effettuare la classificazione (predictions)
predictions = model.predict(inputs_test)

# stampo i valori predictions e i valori ouputs attesi
print("predictions =\n", np.round(predictions, decimals=3))
print("actual =\n", outputs_test)

# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
for index in range(0,len(outputs_test)):
  plt.plot(index, outputs_test[index].argmax(), 'bo', label='Actual')
  plt.plot(index, predictions[index].argmax(), 'r.', label='Predicted')
plt.show()

# Convertiamo la Rete addestrata in un Modello Tensor Flow Lite

Le istruzioni che seguono convertono il modelllo nel formato TFlite. Viene anche stampata la dimensione in byte del modello.

In [ ]:
# Conversione del modello nel formato TensorFlow Lite senza quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Salviamo il modello sul file system
open("gesture_model.tflite", "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Il modello ha una dimensione di %d bytes" % basic_model_size)



## Codifichiamo il Modello (Rete) in in un file Header Arduino

Le istruzioni che seguono creano un array di byte che contiene il modello TFlite.
Il file deve essere importato nello sketch Arduino come nuovo tab.


In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Il file Header, model.h, ha una dimensione di {model_h_size:,} bytes.")
print("\nApri il pannello (effettua evenetualmente il refresh). Effettua il download the file (doppio clic o menu di riga).")

# Classifichiamo i Dati IMU

Ora possiamo eseguire il modello su un Arduino Nano 33 BLE Sense per interpretare i dati dell'accelerometro e del giroscopio.
